In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

brake_df = pd.read_csv('C:\\Users\\jinho\\Desktop\\Taba_AI\\dataset\\0510_brake.csv')
brake_df.head()

,break_value,accel_value,speed,timestamp
0,0,335,0,2024-05-10 07:27:34.730321+00:00
1,6,374,0,2024-05-10 07:27:35.768378+00:00
2,0,345,0,2024-05-10 07:27:36.788696+00:00
3,0,256,2,2024-05-10 07:27:37.809518+00:00
4,0,371,7,2024-05-10 07:27:38.772035+00:00


In [2]:
# 정답값 지정
label = pd.Series(0 for i in range(0, len(brake_df)))

brake_df['target'] = label
brake_df.head()

,break_value,accel_value,speed,timestamp,target
0,0,335,0,2024-05-10 07:27:34.730321+00:00,0
1,6,374,0,2024-05-10 07:27:35.768378+00:00,0
2,0,345,0,2024-05-10 07:27:36.788696+00:00,0
3,0,256,2,2024-05-10 07:27:37.809518+00:00,0
4,0,371,7,2024-05-10 07:27:38.772035+00:00,0


In [3]:
brake_df.drop(labels=['timestamp', 'accel_value'], axis=1, inplace=True)   # 시간 데이터 삭제
brake_df.head()

,break_value,speed,target
0,0,0,0
1,6,0,0
2,0,0,0
3,0,2,0
4,0,7,0


In [4]:
X = brake_df.values[:, :-1] # 학습시킬 특징
Y = brake_df.values[:, -1] # 정답

print(X.shape)
print(Y.shape)


(3080, 2)
(3080,)


In [5]:
# 분할 지점 설정 (예: 처음 80%를 학습 데이터로, 나머지 20%를 테스트 데이터로)
split_index = int(0.8 * len(brake_df))

# 학습 데이터와 테스트 데이터 분리
x_train = X[:split_index]
x_test = X[split_index:]

y_train = Y[:split_index]
y_test = Y[split_index:]


print('Train shape :',x_train.shape)
print('Test shape :',x_test.shape)

Train shape : (2464, 2)
Test shape : (616, 2)


In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # test set에는 transform만 사용하기

In [7]:
import torch

x_train = torch.tensor(x_train)
x_test = torch.tensor(x_test)

# GPU가 사용 가능한 경우 CUDA 장치에 올리기
if torch.cuda.is_available():
    # 기본 CUDA 장치에 텐서 올리기
    x_train = x_train.cuda()
    x_test = x_test.cuda()
    print("Tensor is now on CUDA device:", x_train.device)
else:
    print("CUDA is not available. Tensor remains on CPU:", x_train.device)

ModuleNotFoundError: No module named 'torch'